In [27]:
from mxnet import autograd,nd
import pandas as pd
import numpy as np
data = pd.read_csv("./train.csv")
arr = np.array(data)
arr[:4999,3:6]

array([[281.0, 281.0, 281.0],
       [283.2, 283.2, 283.2],
       [281.4, 281.4, 281.4],
       ...,
       [1538.5, 1553.1, 1532.6],
       [1553.0, 1561.2, 1546.0],
       [1557.4, 1557.7, 1541.3]], dtype=object)

In [28]:
features = arr[:4999,3:]
features = nd.array(features)
features


[[ 281.   281.   281.   282.7    4. ]
 [ 283.2  283.2  283.2  281.1   16. ]
 [ 281.4  281.4  281.4  281.4    0. ]
 ...
 [1538.5 1553.1 1532.6 1561.   238. ]
 [1553.  1561.2 1546.  1563.7  291. ]
 [1557.4 1557.7 1541.3 1556.2  163. ]]
<NDArray 4999x5 @cpu(0)>

In [29]:
labels = arr[1:,6:7]
labels = nd.array(labels)
labels = labels.reshape(shape=[4999])
labels


[ 281.1  281.4  281.9 ... 1563.7 1556.2 1544. ]
<NDArray 4999 @cpu(0)>

In [30]:
from mxnet.gluon import data as gdata

# 超参数
batch_size = 10
dataset = gdata.ArrayDataset(features,labels)
data_iter = gdata.DataLoader(dataset,batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break


[[1242.1  1243.   1236.7  1357.5     6.  ]
 [1333.1  1333.1  1333.1  1336.8     3.  ]
 [ 526.5   538.5   526.5   528.4   995.  ]
 [ 375.1   375.1   375.1   375.1     0.  ]
 [ 882.    891.4   881.2   891.7     6.  ]
 [1236.   1239.25 1236.   1237.3     0.  ]
 [1331.6  1331.6  1331.6  1331.6    61.  ]
 [ 861.6   879.4   861.6   878.     21.  ]
 [ 411.1   411.1   411.1   411.1     2.  ]
 [ 297.3   308.    296.2   297.5  1043.  ]]
<NDArray 10x5 @cpu(0)> 
[1358.5 1327.5  521.   381.7  896.1 1238.2 1324.6  879.5  410.   299.7]
<NDArray 10 @cpu(0)>


In [31]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

In [32]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

In [33]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

In [34]:
from mxnet import gluon
#第一个超参数，学习率，并不是越小越好
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.000000002})

In [35]:
test_data = np.array(pd.read_csv("./test.csv"))
test = nd.array(test_data[0:690,3:])
true_y = test_data[1:691,6:7]
true_y = nd.array(true_y)

In [36]:
def test_acc():
    return loss(net(test),true_y).mean().asnumpy()

In [37]:
num_epochs = 5
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features),labels)
    print('epoch %d,loss: %f test_lost: %f' % (epoch,l.mean().asnumpy(),test_acc()))

epoch 1,loss: 335.722534 test_lost: 816.318115
epoch 2,loss: 255.785858 test_lost: 243.631470
epoch 3,loss: 261.785492 test_lost: 622.565674
epoch 4,loss: 259.000214 test_lost: 477.379181
epoch 5,loss: 255.391022 test_lost: 243.173920


In [38]:
dense = net[0]
dense.weight.data()


[[ 2.6279724e-01  2.4579675e-01  2.4257064e-01  2.5183904e-01
  -5.6270343e-05]]
<NDArray 1x5 @cpu(0)>

In [39]:
dense.bias.data()


[0.00020979]
<NDArray 1 @cpu(0)>